## CASSANDRA ASSIGNMENT_Gopi Chand Yenikapalli

### Import packages

In [1]:
import pandas as pd
from cassandra.cluster import Cluster

### Create a session connection to Cassandra cluster

In [2]:
clstr = Cluster()
session = clstr.connect()

### Use session to 'talk' to cassandra

In [3]:
session.execute("DROP KEYSPACE IF EXISTS m14")

In [4]:
rows = session.execute("desc keyspaces")
for row in rows:
    print(f"{row[0]}")

system
system_auth
system_distributed
system_schema
system_traces
system_views
system_virtual_schema


In [5]:
session.execute("CREATE KEYSPACE IF NOT EXISTS m14 WITH REPLICATION = {'class':'SimpleStrategy', 'replication_factor':2}")

In [6]:
rows = session.execute("desc keyspaces")
for row in rows:
    print(f"{row[0]}")

m14
system
system_auth
system_distributed
system_schema
system_traces
system_views
system_virtual_schema


Create a inventory table for testing

In [7]:
session.execute("""
CREATE TABLE IF NOT EXISTS m14.inventory (
    sku TEXT, 
    name TEXT, 
    description TEXT, 
    warehouse_num INT,
    PRIMARY KEY(sku, warehouse_num)
);
""")

In [8]:

# Sample data to insert
sample_data = [
    ("SKU001", "TV", "led tv 25 inch curved edge", 1),
    ("SKU002", "headphone", "headphone pro3, ", 1),
    ("SKU003", "toolkit", "toolkit driver 12 set", 3),
    ("SKU004", "cycle", "cycle adult racing", 3),
    ("SKU005", "clothe", "men 32 slimfit", 2),
    ("SKU006", "blub", "light 32w daylight", 3),
    ("SKU007", "laptop", "dell 16gb i7", 1),
    ("SKU008", "water", "12pack 1L", 4),
    ("SKU009", "mobile", "iphone 14 promax", 2)
]

# Insert the data
for entry in sample_data:
    insert_query = """
    INSERT INTO m14.inventory (sku, name, description, warehouse_num)
    VALUES (%s, %s, %s, %s)
    """
    session.execute(insert_query, entry)


In [9]:
# Query to fetch all records from store_inventory
query = "SELECT * FROM m14.inventory"
rows = session.execute(query)

# Print the data
print("SKU\t\tName\t\tDescription\t\tWarehouse Number")
print("-" * 80)  # just a separator line
for row in rows:
    print(f"{row.sku}\t\t{row.name}\t\t{row.description}\t\t{row.warehouse_num}")


SKU		Name		Description		Warehouse Number
--------------------------------------------------------------------------------
SKU005		clothe		men 32 slimfit		2
SKU002		headphone		headphone pro3, 		1
SKU008		water		12pack 1L		4
SKU006		blub		light 32w daylight		3
SKU004		cycle		cycle adult racing		3
SKU001		TV		led tv 25 inch curved edge		1
SKU003		toolkit		toolkit driver 12 set		3
SKU009		mobile		iphone 14 promax		2
SKU007		laptop		dell 16gb i7		1


In [10]:
rows = session.execute("SELECT (sku, name, description, warehouse_num) FROM m14.inventory")
for row in rows:
    print(f"sku={row[0][0]}, name={row[0][1]}, description={row[0][2]}, warehouse_num={row[0][3]}")

sku=SKU005, name=clothe, description=men 32 slimfit, warehouse_num=2
sku=SKU002, name=headphone, description=headphone pro3, , warehouse_num=1
sku=SKU008, name=water, description=12pack 1L, warehouse_num=4
sku=SKU006, name=blub, description=light 32w daylight, warehouse_num=3
sku=SKU004, name=cycle, description=cycle adult racing, warehouse_num=3
sku=SKU001, name=TV, description=led tv 25 inch curved edge, warehouse_num=1
sku=SKU003, name=toolkit, description=toolkit driver 12 set, warehouse_num=3
sku=SKU009, name=mobile, description=iphone 14 promax, warehouse_num=2
sku=SKU007, name=laptop, description=dell 16gb i7, warehouse_num=1


In [11]:
rows = session.execute("SELECT (sku, name, description, warehouse_num) FROM m14.inventory WHERE warehouse_num = 1 ")
for row in rows:
    print(f"{row[0][0]}, {row[0][1]}, {row[0][2]}, {row[0][3]}")

InvalidRequest: Error from server: code=2200 [Invalid query] message="Cannot execute this query as it might involve data filtering and thus may have unpredictable performance. If you want to execute this query despite the performance unpredictability, use ALLOW FILTERING"

The above query will generate an error. You can make this work by allowing filtering, but this is not a good idea.

In [12]:
rows = session.execute("SELECT (sku, name, description, warehouse_num) FROM m14.inventory WHERE warehouse_num = 1 ALLOW FILTERING")
for row in rows:
    print(f"{row[0][0]}, {row[0][1]}, {row[0][2]}, {row[0][3]}")

SKU002, headphone, headphone pro3, , 1
SKU001, TV, led tv 25 inch curved edge, 1
SKU007, laptop, dell 16gb i7, 1


In [13]:
# Create an index on warehouse_num in the inventory table
session.execute("""
CREATE INDEX IF NOT EXISTS warehouse_num
ON m14.inventory (warehouse_num);
""")

In [14]:
rows = session.execute("SELECT (sku, name, description, warehouse_num) FROM m14.inventory WHERE warehouse_num = 1 ")
for row in rows:
    print(f"{row[0][0]}, {row[0][1]}, {row[0][2]}, {row[0][3]}")

SKU002, headphone, headphone pro3, , 1
SKU001, TV, led tv 25 inch curved edge, 1
SKU007, laptop, dell 16gb i7, 1
